In [165]:
import pandas as pd
import numpy as np
import difflib
from math import cos, radians
from haversine import haversine, Unit
import folium




In [166]:
#INPUT CELL:
input_recipe = input("Please enter a recipe (e.g. Pasta Putanesca): ")
Lat_in = 35.2220
Lon_in = -101.8313


Please enter a recipe (e.g. Pasta Putanesca): chicken fontina


In [167]:
farm_in = "USDA_file.csv"
recipe_in = "Recipe_file.csv"
farms = pd.read_csv(farm_in)
recipes = pd.read_csv(recipe_in)


In [168]:
recipes['title'] = recipes['title'].astype(str)
recipes['matched_ingredient'] = recipes['matched_ingredient'].astype(str)
farms['LAT'] = farms['LAT'].astype(float)
farms['LON'] = farms['LON'].astype(float)

In [169]:
rec_list = recipes.title.unique().tolist()
hey = rec_list[:1000]
match_list = difflib.get_close_matches(input_recipe, rec_list, n=3, cutoff=0.1)


In [170]:
match_df = recipes[recipes['title'].isin(match_list)]
ingredient_list = match_df.matched_ingredient.unique().tolist()


In [171]:
farms['INGREDIENT']=farms['INGREDIENT'].apply(lambda x: x.lower())

In [172]:
farm_search = farms[farms['INGREDIENT'].isin(ingredient_list)]


In [173]:


farm_search.loc[:,'es'] = farm_search.apply(lambda x: haversine((Lat_in,Lon_in),(x.LAT, x.LON),unit='mi'), axis=1)



/Users/networkadministrator/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/networkadministrator/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [174]:
def my_agg(x):
    names = {
        'score': x['es'].min()}

    return pd.Series(names, index=['score'])

idx = farm_search.groupby(['INGREDIENT'])['es'].transform(min) == farm_search['es']
farm_vals = farm_search[idx]
alpha = farm_vals.groupby(['INGREDIENT','LAT','LON']).apply(my_agg).reset_index()
beta = alpha.set_index(['INGREDIENT'])



In [175]:
def score_ex(x,y):
    try:
        return beta.loc[x][y]
    except:
        return None

match_df.loc[:,'score'] = match_df.loc[:,'matched_ingredient'].apply(lambda x: score_ex(x,'score'))
match_df.loc[:,'Lat'] = match_df.loc[:,'matched_ingredient'].apply(lambda x: score_ex(x,'LAT'))
match_df.loc[:,'Lon'] = match_df.loc[:,'matched_ingredient'].apply(lambda x: score_ex(x,'LON'))


In [176]:
#UPDATES: grabbing original and grouping later to show all ingredients at given location

scores = match_df.groupby('title').score.agg(['mean']).sort_values('mean', ascending=True).reset_index()
winner = scores.title.iloc[0]

closest_df = match_df[match_df['title']==winner][['original','Lat','Lon']]
closest_2 = closest_df

closest_2

,original,Lat,Lon
114703,3 tablespoons olive oil,32.217942,-97.769211
114704,1 3 1/2 pound chicken,35.398675,-101.893804
114705,6 garlic cloves,34.962529,-101.895547
114706,1 1/2 teaspoon aniseed,34.572984,-103.346055
114707,1 28-ounce can Italian plum tomatoes,34.962529,-101.895547
114708,1/2 cup low-salt chicken broth,35.398675,-101.893804
114709,1/4 cup Ouzo,NaN,NaN
114710,1 tablespoon dried oregano,34.572984,-103.346055
114711,12 black brine-cured olives,32.217942,-97.769211
114712,4 ounces feta cheese,NaN,NaN


In [177]:
#UPDATES: here is the groupby

closest_df = closest_df.dropna().reset_index()[['original','Lat','Lon']].reset_index()

closest_frame = closest_df.groupby(['Lat', 'Lon']).original.apply(list).reset_index()


In [178]:
frame_in = pd.DataFrame({'original': ['Home'], 'Lat': [Lat_in], 'Lon': [Lon_in]})


In [179]:

         
m=folium.Map(location=[Lat_in,Lon_in])
for item in range(len(closest_frame)):
    df_new = pd.concat([closest_frame.iloc[[item]], frame_in], sort=False)
    
    points = list(df_new[['Lat','Lon']].to_records(index=False))
    #print(df_new["position_lat_degrees"].head())
    
    try:
        folium.PolyLine(locations=points,weight=5).add_to(m)
        folium.Marker([df_new["Lat"].iloc[0],df_new["Lon"].iloc[0]], 
                      popup=df_new['original'].iloc[0]).add_to(m)
        
    except: 
        print('Farm to Face Map has been Canned')
    #m.add_children(route)
m.fit_bounds([[closest_df.Lat.min(), closest_df.Lon.min()], [closest_df.Lat.max(), closest_df.Lon.max()]])
display(m)
print('The closest matching recipe is: ' + winner)
print(str(len(closest_2.dropna())) + ' of ' + str(len(closest_2)) + ' ingredients located')



The closest matching recipe is: Chicken Athena 
8 of 10 ingredients located
